In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.MatchLSTM.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6276.37it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3859.54it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2484280.89it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7846.15it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3950.78it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 740342.34it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8319.46it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f1bc5facc50>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f1bc5face48>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10,
    batch_size=20,
    resample=True,
    sort=False,
    shuffle=True
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20,
    sort=False,
    shuffle=False
)

In [9]:
padding_callback = mz.models.MatchLSTM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.MatchLSTM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding'] = embedding_matrix

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchLSTM(
  (embedding): Embedding(30058, 300, padding_idx=0)
  (dropout): Dropout(p=0.2, inplace=False)
  (input_proj): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(300, 100, bidirectional=True)
    )
  )
  (match_module): MatchModule(
    (v2_proj): Linear(in_features=200, out_features=200, bias=True)
    (proj): Linear(in_features=800, out_features=400, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (mlstm_module): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(400, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=200, out_features=200, bias=True)
    (2): Tanh()
  )
  (out): Linear(in_features=200, out_features=1, bias=True)
)
Trainable params:  10141601


In [11]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()

[Iter-255 Loss-2.017]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5984 - normalized_discounted_cumulative_gain@5(0.0): 0.6485 - mean_average_precision(0.0): 0.6037



[Iter-510 Loss-0.620]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5185 - normalized_discounted_cumulative_gain@5(0.0): 0.5709 - mean_average_precision(0.0): 0.5247



[Iter-765 Loss-0.109]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5035 - normalized_discounted_cumulative_gain@5(0.0): 0.5765 - mean_average_precision(0.0): 0.5264



[Iter-1020 Loss-0.050]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4745 - normalized_discounted_cumulative_gain@5(0.0): 0.5447 - mean_average_precision(0.0): 0.5009



[Iter-1275 Loss-0.051]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.515 - normalized_discounted_cumulative_gain@5(0.0): 0.5844 - mean_average_precision(0.0): 0.5369



[Iter-1530 Loss-0.039]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4998 - normalized_discounted_cumulative_gain@5(0.0): 0.5532 - mean_average_precision(0.0): 0.5154



[Iter-1785 Loss-0.033]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5207 - normalized_discounted_cumulative_gain@5(0.0): 0.5816 - mean_average_precision(0.0): 0.5363



[Iter-2040 Loss-0.024]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5133 - normalized_discounted_cumulative_gain@5(0.0): 0.5682 - mean_average_precision(0.0): 0.5301



[Iter-2295 Loss-0.028]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5321 - normalized_discounted_cumulative_gain@5(0.0): 0.6002 - mean_average_precision(0.0): 0.5494

